# Morphing

> This module apply morphing to a template source from landmarks. This file is the script to run and call from Command Line. 


 -  Install the repo with pip. 
 -  Download with git clone infepy. 
 -  Load you data in the data folder. 

The *Source* is the template model, the geometry is stored in .key file and the landmarks in .fcsv/.csv file.
The target folder will be used for the file to morph. 
If there is only one target, place the landamarks inside the target folder.

## Target folder structure:
### One target:

 -  ../test_data/target/landmarks_target.fcsv"

    In the config file:


            path = "../test_data/target"
            filename_landmarks = "landmarks_target.fcsv"


### Multiple targets:

-       
    ../test_data/target/subj1/landmarks.fcsv

    ../test_data/target/subj2/landmarks.fcsv

    ../test_data/target/subj3/landmarks.fcsv

    ../test_data/target/subj4/landmarks.fcsv

        In the config file:

           path = "data/target"
           
           filename_landmarks = "landmarks_target.fcsv"

- From the command line go into infepy/infepy.

        - type: python morph.py

        The module can be called with different options.
        
        - morph.py [-h] [--function FUNCTION] [--smoothing SMOOTHING]

        optional arguments:
        - -h, --help            show this help message and exit
        -  --function FUNCTION   Basis Function for morphing. Available function from rbf_factory: thin_plate_spline, euclidean, gaussian_spline, multi_quadratic_biharmonic_spline, inv_multi_quadratic_biharmonic_spline, beckert_wendland_c2_basis, polyharmonic_spline
        -  --smoothing SMOOTHING Smoothing Factor


### For General Procrustes Analysis (GPA) / Principal Components Analysis (PCA):
- follow the tutorials:
  - [Tutorial GPA1](https://github.com/SlicerMorph/Tutorials/tree/main/GPA_1)
  - [Tutorial GPA2](https://github.com/SlicerMorph/Tutorials/blob/main/GPA_2/README.md)

-----------------------------------------------------------

-----------------------------------------------------------

In [ ]:
#| default_exp morph

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import numpy as np
import click
from argparse import ArgumentParser

In [ ]:
#| export
import infepy.rbf as rbf
import infepy.preprocessing as pre
import infepy.utils as utils

In [ ]:
#| export
def morphing(source_landmarks: np.ndarray,  # Landmarks of the source mesh
             target_landmarks: np.ndarray,  # Target Landmarks
             source_mesh: np.ndarray, # Coordinates of the mesh/ mesh 
             func,
             smoothing):
    "Morph the target mesh with RBF function - Thin Plate Spine."
    basis_function = func
    
    rad_bas_fun = rbf.RBF(original_control_points=utils.from_df_to_np(source_landmarks) , 
              deformed_control_points= utils.from_df_to_np(target_landmarks),
              func= basis_function,
              smoothing= smoothing)

    return rad_bas_fun(utils.from_df_to_np(source_mesh))

## Command Line

In [ ]:
#| export
parser = ArgumentParser(description="Morphing function")
parser.add_argument("--function", 
                    type=str,
                    default='thin_plate_spline',
                    help="Basis Function for morphing")
parser.add_argument("--smoothing", 
                    type=float,
                    default=0.0,
                    help="Smoothing Factor")
        
def do_morphing(func, smoothing):
    config = utils.read_toml()
    source_landmarks = pre.read_landmarks(utils._merge_path(config['source']['path'],config['source']['filename_landmarks']))
    template_mesh = pre.read_nodes(utils._merge_path(config['source']['path'],config['source']['filename_mesh']))
    
    if utils.multiple_targets() == False: # single target
        target_landmarks = pre.read_landmarks(utils._merge_path(config['target']['path'],config['target']['filename_landmarks']))
        pre._check_landmarks(source_landmarks, target_landmarks)
        morphed_mesh = morphing(source_landmarks, target_landmarks, template_mesh,func, smoothing)
        pre.write_output(morphed_mesh,
                     utils._merge_path(config['target']['path'], 'morphed.key'),
                     utils._merge_path(config['source']['path'], config['source']['filename_mesh']))
    else:
        targets_folder = utils.multiple_targets()
        for folder in targets_folder:
            folder_path = os.path.join(config['target']['path'],folder)
            target_landmarks = pre.read_landmarks(utils._merge_path(folder_path,config['target']['filename_landmarks']))
            pre._check_landmarks(source_landmarks, target_landmarks)
            morphed_mesh = morphing(source_landmarks, target_landmarks, template_mesh,func, smoothing)
            pre.write_output(morphed_mesh,
                      utils._merge_path(folder_path,'morphed.key'),
                      utils._merge_path(config['source']['path'],config['source']['filename_mesh']))
    return

In [ ]:
#| hide
do_morphing(func='thin_plate_spline', smoothing=None)

../test_data/Config.toml
../test_data/Config.toml
../test_data/Config.toml


/mnt/c/Users/chiarar/OneDrive - Chalmers/SSMT - Subject Specific Modeling Toolstack/Github/infepy/infepy/rbf_factory.py:111: RuntimeWarning: divide by zero encountered in log
  result = np.where(arg > 0, result * np.log(arg), result)
/mnt/c/Users/chiarar/OneDrive - Chalmers/SSMT - Subject Specific Modeling Toolstack/Github/infepy/infepy/rbf_factory.py:111: RuntimeWarning: invalid value encountered in multiply
  result = np.where(arg > 0, result * np.log(arg), result)
/mnt/c/Users/chiarar/OneDrive - Chalmers/SSMT - Subject Specific Modeling Toolstack/Github/infepy/infepy/rbf_factory.py:111: RuntimeWarning: divide by zero encountered in log
  result = np.where(arg > 0, result * np.log(arg), result)
/mnt/c/Users/chiarar/OneDrive - Chalmers/SSMT - Subject Specific Modeling Toolstack/Github/infepy/infepy/rbf_factory.py:111: RuntimeWarning: invalid value encountered in multiply
  result = np.where(arg > 0, result * np.log(arg), result)
/mnt/c/Users/chiarar/OneDrive - Chalmers/SSMT - Subject 

In [ ]:
#|eval: false
#|export
if __name__ == "__main__":
    args= parser.parse_args()
    print("Radial Basis Function: ", args.function)
    print("Smoothing factor: ", args.smoothing)
    do_morphing(args.function, args.smoothing)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()